In [76]:
from datasets import load_dataset, Audio
import torch
from transformers import EncodecModel, AutoProcessor
from scipy.io import wavfile
import numpy as np
import librosa

In [ ]:
#https://colab.research.google.com/github/Vaibhavs10/notebooks/blob/main/use_encodec_w_transformers.ipynb#scrollTo=Lguq4cjBNjrI

In [4]:
model = EncodecModel.from_pretrained("facebook/encodec_24khz")
processor = AutoProcessor.from_pretrained("facebook/encodec_24khz")

/home/kuro/miniconda3/envs/latest/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [85]:
# Read the WAV file
audio_data, sample_rate = librosa.load('./audio.wav', sr=processor.sampling_rate)

print(sample_rate, processor.sampling_rate)

# Convert the audio data to a float32 NumPy array
audio_array = audio_data.astype(np.float32)

import IPython.display as ipd

ipd.Audio(audio_array, rate=processor.sampling_rate)

24000 24000


In [101]:
inputs = processor(raw_audio=audio_array, sampling_rate=processor.sampling_rate, return_tensors="pt")
encoder_outputs = model.encode(inputs["input_values"], inputs["padding_mask"], bandwidth = 6.0)

In [102]:
len(encoder_outputs.audio_codes[0][0][0])

725

In [103]:
audio_values = model.decode(encoder_outputs.audio_codes, encoder_outputs.audio_scales, inputs["padding_mask"])[0]

In [104]:
print(audio_values.detach().numpy()[0])

[[ 2.5448487e-03  1.9008785e-03  2.6601371e-03 ... -7.3055868e-05
   2.4556805e-04  8.0547610e-04]]


In [105]:
import IPython.display as ipd

ipd.Audio(audio_values.detach().numpy()[0][0], rate=processor.sampling_rate)